In [249]:
import pandas as pd
import numpy as np
import os
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score, make_scorer
from sklearn.tree import DecisionTreeClassifier as dtc 
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [377]:
log_directory_tls12 = "/home/admin/Test_dir/datasets/ms/TLS1.2/short_cert/"
log_files12 = [f for f in os.listdir(log_directory_tls12) if f.endswith('.log')]

log_directory_tls13 = "/home/admin/Test_dir/datasets/ms/TLS1.3/"
log_files13 = [f for f in os.listdir(log_directory_tls13) if f.endswith('.log')]

df_list = []

for log_file in log_files12:
    file_path = os.path.join(log_directory_tls12, log_file)
    df = pd.read_csv(file_path, sep=",", header=None, skiprows=8, low_memory=False)
    df = df.iloc[:-1]
    df.replace("-", pd.NA, inplace=True)
    df = df.dropna()
    df_list.append(df)

for log_file in log_files13:
    file_path = os.path.join(log_directory_tls13, log_file)
    df = pd.read_csv(file_path, sep=",", header=None, skiprows=8, low_memory=False)
    df = df.iloc[:-1]
    df.replace("-", pd.NA, inplace=True)
    df = df.dropna()
    df_list.append(df)



headers = ['client_hello_len','client_hello_ext_num','server_hello_len','server_hello_ext_num','tls_version']
df_ms = pd.concat(df_list, ignore_index=True)
df_ms.shape
df_ms = shuffle(df_ms, random_state=42)
df_ms.reset_index(drop=True, inplace=True)
# df_ms = df_ms.iloc[:40000]
df_ms = df_ms.set_axis(headers, axis=1)
df_ms['c2'] = 1
df_ms['source'] = 'MS'
df_ms_noise = df_ms.iloc[40001:42000]
df_ms = df_ms.iloc[:40000]
df_ms_noise.loc[:, 'c2'] = 0

In [378]:
log_directory = "/home/admin/Test_dir/datasets/tranco"

# List all .log files in the directory
log_files = [f for f in os.listdir(log_directory) if f.endswith('.log')]

# List to hold the DataFrames
df_list = []

# Loop through each log file and load it into a DataFrame
for log_file in log_files:
    file_path = os.path.join(log_directory, log_file)
    
    # Read the log file into a DataFrame (assuming CSV or tab-delimited format)
    df = pd.read_csv(file_path, sep=",", header=None, skiprows=8, low_memory=False)  # or change delimiter depending on the log format
    df = df.iloc[:-1]
    df.replace("-", pd.NA, inplace=True)
    # Remove rows where any column contains NaN (or previously "-")
    df = df.dropna()
    # Append the DataFrame to the list
    df_list.append(df)

#Load Tranco dataset (i.e., Benign traffic)
df_tranco= pd.concat(df_list, ignore_index=True)
df_tranco = shuffle(df_tranco, random_state=42)
df_tranco.reset_index(drop=True, inplace=True)
df_tranco = df_tranco.iloc[:100000]
df_tranco = df_tranco.set_axis(headers, axis=1)
# df_tranco = df_tranco[headers[0:3]]
# df_tranco = df_tranco[['tls_b_0','tls_b_1','tls_b_5','tls_b_7']]
df_tranco['c2'] = 0
df_tranco['source'] = 'tranco'

In [379]:
log_directory_benign = "/home/admin/Test_dir/datasets/DoHBrw/benign"

# List all .log files in the directory
log_files_benign = [f for f in os.listdir(log_directory_benign) if f.endswith('.log')]

# List to hold the DataFrames
df_list = []

# Loop through each log file and load it into a DataFrame
for log_file in log_files_benign:
    file_path = os.path.join(log_directory_benign, log_file)
    
    # Read the log file into a DataFrame (assuming CSV or tab-delimited format)
    df = pd.read_csv(file_path, sep=",", header=None, skiprows=8, low_memory=False)  # or change delimiter depending on the log format
    df = df.iloc[:-1]
    df.replace("-", pd.NA, inplace=True)
    # Remove rows where any column contains NaN (or previously "-")
    df = df.dropna()
    # Append the DataFrame to the list
    df_list.append(df)

#Load Tranco dataset (i.e., Benign traffic)
df_doh_benign= pd.concat(df_list, ignore_index=True)
df_doh_benign = shuffle(df_doh_benign, random_state=42)
df_doh_benign.reset_index(drop=True, inplace=True)
df_doh_benign = df_doh_benign.iloc[:20000]
df_doh_benign = df_doh_benign.set_axis(headers, axis=1)
df_doh_benign['c2'] = 0
df_doh_benign['source'] = 'DoH'
###########################################################################################################################
log_directory_malicious = "/home/admin/Test_dir/datasets/DoHBrw/malicious/"

# List all .log files in the directory
log_files_malicious = [f for f in os.listdir(log_directory_malicious) if f.endswith('.log')]

# List to hold the DataFrames
df_list = []

# Loop through each log file and load it into a DataFrame
for log_file in log_files_malicious:
    file_path = os.path.join(log_directory_malicious, log_file)
    
    # Read the log file into a DataFrame (assuming CSV or tab-delimited format)
    df = pd.read_csv(file_path, sep=",", header=None, skiprows=8, low_memory=False)  # or change delimiter depending on the log format
    df = df.iloc[:-1]
    df.replace("-", pd.NA, inplace=True)
    # Remove rows where any column contains NaN (or previously "-")
    df = df.dropna()
    # Append the DataFrame to the list
    df_list.append(df)

#Load Tranco dataset (i.e., Benign traffic)
df_doh_malicious= pd.concat(df_list, ignore_index=True)
df_doh_malicious = shuffle(df_doh_malicious, random_state=42)
df_doh_malicious.reset_index(drop=True, inplace=True)
df_doh_malicious = df_doh_malicious.iloc[:20000]
df_doh_malicious = df_doh_malicious.set_axis(headers, axis=1)
df_doh_malicious['c2'] = 1
df_doh_malicious['source'] = 'DoH'

df_doh = pd.concat([df_doh_malicious,df_doh_benign])

In [380]:
# # #Load MTA dataset (i.e., Malicious and Benign traffic)
# df_mta = pd.read_csv('/home/admin/extending12to13/processed-datasets/labeled_mta.csv', low_memory=False)
# columns = ['tls_b_0','tls_b_1','tls_b_5','tls_b_7'] + ["c2"]
# df_mta = df_mta[columns]
# # df_mta = df_mta[df_mta["tls_b_9"]>0]
# df_mta['source'] = "MTA"
# df_mta.reset_index(drop=True, inplace=True)

In [381]:
df_all = pd.concat([df_tranco,df_ms,df_ms_noise, df_doh], ignore_index=True)

In [382]:
df_all = shuffle(df_all, random_state=42)

In [383]:
df_all.reset_index(drop=True, inplace=True)

# Split Data

In [384]:
X = df_all.loc[:,headers].values.astype(np.float64)
y = df_all.loc[:,"c2"].values
y

array([0, 1, 1, ..., 1, 0, 0])

In [385]:
X_train, X_test, y_train, y_test = train_test_split(X,y , 
                                   random_state=42,  
                                   test_size=0.25,  
                                   shuffle=True) 

# Training

In [386]:
scoring = {"AUC": "roc_auc",
           "Accuracy": "accuracy",
           "Balanced Accracy": "balanced_accuracy",
           "Recall": "recall",
           "Neg Recall": make_scorer(recall_score, pos_label=0),
           "F1": "f1"}

In [387]:
dtc_classifier = dtc(random_state=42)
parameters = {'max_depth':list(range(1,20))+[30,40,50,60], 
              'criterion': ['gini', 'entropy'], 
              'splitter': ['best', 'random'], 
              'max_features': [None, 'sqrt', 'log2'],
              'class_weight': [None, 'balanced']}

In [388]:
# grid_search = GridSearchCV(estimator=dtc_classifier, param_grid=parameters, cv=5)
# grid_search.fit(X_train, y_train)
# print("Best parameters found: ", grid_search.best_params_)
# print("Best parameters found: ", grid_search.best_params_)
# tree_model = grid_search.best_estimator_
# tree_model.fit(X_train, y_train)
# y_pred = tree_model.predict(X_test)
# accuracy = accuracy_score(y_test, y_pred)
# print(f"Accuracy on test data: {accuracy}")
# importances = tree_model.feature_importances_
# feature_importances = pd.DataFrame({'feature': headers, 'importance': importances})
# feature_importances = feature_importances.sort_values('importance', ascending=False)
# feature_importances

In [391]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
rf_model = RandomForestClassifier(max_depth=20, n_estimators = 5, max_leaf_nodes=500, n_jobs=4, random_state=42, bootstrap=False)                              
rf_model.fit(X_train, y_train)
y_pred = rf_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy on test data: {accuracy}")
importances = rf_model.feature_importances_
feature_importances = pd.DataFrame({'feature': headers, 'importance': importances})
feature_importances = feature_importances.sort_values('importance', ascending=False)
feature_importances

Accuracy on test data: 0.9862043035631652


,feature,importance
1,client_hello_ext_num,0.513539
0,client_hello_len,0.210550
3,server_hello_ext_num,0.209984
4,tls_version,0.059575
2,server_hello_len,0.006352


In [392]:
# import pickle
# pickle.dump(rf_model, open("RF.pkl", 'wb'))